<a href="https://colab.research.google.com/github/Vigneshthanga/258-Deep-Learning/blob/master/Ungraded5/SaveLoadModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install H5PY

In [5]:
!pip install pyyaml h5py

## Import Libraries

In [6]:
import os
import tensorflow as tf
from tensorflow import keras
print(tf.version.VERSION)

2.2.0


In [0]:
(xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.mnist.load_data()
ytrain = ytrain[:1000]
ytest = ytest[:1000]
xtrain = xtrain[:1000].reshape(-1, 28 * 28) / 255.0
xtest = xtest[:1000].reshape(-1, 28 * 28) / 255.0

## Model Definition

In [8]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [9]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(xtrain, 
          ytrain,  
          epochs=10,
          validation_data=(xtest,ytest),
          callbacks=[cp_callback])  # Pass callback to training

Epoch 1/10
21/32 [==================>...........] - ETA: 0s - loss: 1.4567 - accuracy: 0.5714
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 1.2003 - accuracy: 0.6630 - val_loss: 0.7351 - val_accuracy: 0.7770
Epoch 2/10
26/32 [=======================>......] - ETA: 0s - loss: 0.4310 - accuracy: 0.8798
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.4220 - accuracy: 0.8840 - val_loss: 0.5318 - val_accuracy: 0.8310
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2742 - accuracy: 0.9340
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2857 - accuracy: 0.9330 - val_loss: 0.4851 - val_accuracy: 0.8470
Epoch 4/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2246 - accuracy: 0.9442
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [10]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [11]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(xtest,  ytest, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4063 - accuracy: 0.0990
Untrained model, accuracy:  9.90%


In [12]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(xtest,  ytest, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3977 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## Adding checkpoint callback functions

In [13]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(xtrain, 
          ytrain,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(xtest,ytest),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [14]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

## Model Definition

In [16]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(xtest,  ytest, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4985 - accuracy: 0.8770
Restored model, accuracy: 87.70%


## Save the model using checkpoint callback

In [17]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(xtest,  ytest, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4985 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(xtrain, ytrain, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1793 - accuracy: 0.6600
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4356 - accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2847 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2068 - accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1600 - accuracy: 0.9620
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


## Save the model

In [19]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Model Evaluation

In [21]:
# Evaluate the restored model
loss, acc = new_model.evaluate(xtest,  ytest, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(xtest).shape)

32/32 - 0s - loss: 0.4169 - accuracy: 0.8610
Restored model, accuracy: 86.10%
(1000, 10)


In [22]:
# Create and train a new model instance.
model = create_model()
model.fit(xtrain, ytrain, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1669 - accuracy: 0.6600
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4130 - accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2819 - accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2104 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1430 - accuracy: 0.9740


## Load and evaluate the model from H5 file

In [23]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss, acc = new_model.evaluate(xtest,  ytest, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4201 - accuracy: 0.8660
Restored model, accuracy: 86.60%
